In [ ]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))
# sys.path

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport optiml.queries
%matplotlib inline

In [ ]:
from optiml.connection import SnowflakeConnConfig
from optiml.queries import SNFLKQuery
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
qlib = SNFLKQuery(connection, 'KIV')

# Total cost breakdown

## Visualization

In [ ]:
df = qlib.total_cost_breakdown('2022-10-01', '2022-10-31')
df

In [ ]:
df.dtypes

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "table"},{"type": "pie"}]]
)
# fig = go.Figure()

fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['dollars'].tolist(),name="Dollars"),row=1,col=1)
fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['credits'].tolist(),name='Credits'),row=1,col=2)

# fig.add_trace(go.Table(header=dict(values=['cost_category','credits', 'dollars'],font_size=10),
#                   cells=dict(values=[df['cost_category'],df['credits'],df['dollars']],font_size=10)),row=1, col=1)
#fig.update_layout(font_size=12)
fig.update_layout(
    title={
        'text': "Breakdown of total cost by usage category",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## Analysis
- Compute forms overwhelming majority of the cost
- Cloud services are in aggregate < 10 % of compute so are not an added expense
- Autoclustering used to be on in 2020 but isn't anymore
- Should explore if there are any compute efficiencies to be gained by 
    - Autoclustering
    - Materialized views / caching
    - Search optimization
    - Query optimizations / reducing scans

* Largely used by compute
* In 2020 auto clustering was on but was turned off - why?
* Storage credits at 0 since storage is charged as a flat rate - are numbers consistent?
* Definitely did not use $80K worth of credits in October since total bill is $50K - so something is wrong!


In [ ]:
df = qlib.cost_of_compute_ts('2022-10-12', '2022-10-13')

In [ ]:
df

In [ ]:
df['TOTAL_CREDITS_USED'].sum()